In [ ]:
import os
import csv
import numpy as np
import torch
import utils
import torchopenl3

In [ ]:
# GPU
n_gpu = torch.cuda.device_count()
print("n_gpu: ", n_gpu)

os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="3"

if n_gpu == 0:
    print("Warning: There\'s no GPU available on this machine")
    device = 'cpu'
else:
    print("Visible devices: ", os.environ["CUDA_VISIBLE_DEVICES"])
    device = 'cuda:0'
    torch.set_default_tensor_type('torch.cuda.FloatTensor')
    print("Default tensor type set to torch.cuda.FloatTensor")

print("Device: ", device)

In [ ]:
# samples_path = '../../_data/zapsplat_pack_footsteps_high_heels_1s_aligned'
samples_path = '../../_data/zapsplat_misc_shoes_misc_surfaces_inception_network'
# samples_path = '../checkpoints/2021-09-20_19h46m/120k_generated_audio_large'
embs_folder = "embeddings/real"
csv_filename = 'openl3_embeddings_zapsplat_misc_shoes_misc_surfaces_inception_network.csv'
if not os.path.exists(f'{embs_folder}'):
    os.makedirs(f'{embs_folder}')

input_repr = 'mel256'   # "linear", "mel128" (default), "mel256"
content_type = 'env'    # “env” (environmental), “music” (default)
embedding_size = 512    # 512, 6144 (default)

In [ ]:
# get embeddings and export to csv
model = torchopenl3.models.load_audio_embedding_model(
                            input_repr=input_repr, 
                            content_type=content_type,
                            embedding_size=embedding_size
                            )
openl3_embs = utils.get_openl3_embs_subfolders(model, path=samples_path)

In [6]:
# write to csv
embs_to_csv = []
for row in openl3_embs:
    emb = [row[0], row[1]]
    for el in row[2]:
        emb.append(el)
    embs_to_csv.append(emb)

fields = ['class', 'filename']
for i in range(512):
    fields.append(f'openl3_{i}')

with open(f'{embs_folder}/{csv_filename}', 'w') as f:
    write = csv.writer(f)
    write.writerow(fields)
    write.writerows(embs_to_csv)